<a href="https://colab.research.google.com/github/alexacheui/ateema_capstone/blob/main/%5BVersion_2%5D_Deterministic_classed.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [12]:
import os
os.chdir('/content/drive/My Drive/ADSP Capstone/Ateema')

In [13]:
!pwd

/content/drive/My Drive/ADSP Capstone/Ateema


In [14]:
# extract title with url

import re
import pandas as pd

def read_rtf(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        content = file.read()

    # Remove RTF formatting tags
    content = re.sub(r'\\[a-zA-Z]+\d*', '', content)  # Remove commands
    content = re.sub(r'{|}', '', content)             # Remove braces
    content = re.sub(r'\\\*\\[^ ]+ ', '', content)    # Remove special characters
    content = re.sub(r'\n|\r', '', content)           # Remove newlines

    return content

def extract_urls_from_rtf(content):
    # Regular expressions to extract title and URL
    title_pattern = r'"Title": "(.*?)"'
    url_pattern = r'"URL": "(.*?)"'

    titles = re.findall(title_pattern, content)
    urls = re.findall(url_pattern, content)

    # Filter out entries with "No URL available"
    filtered_data = {"Title": [], "URL": []}
    for title, url in zip(titles, urls):
        if url != "No URL available":
            filtered_data["Title"].append(title)
            filtered_data["URL"].append(url)

    # Create a DataFrame
    df = pd.DataFrame(filtered_data)
    return df

# Path to the RTF file
file_path = 'Final_Output.rtf'

# Read the RTF file and clean the content
text_content = read_rtf(file_path)

# Extract URLs and create DataFrame
df = extract_urls_from_rtf(text_content)

# Display DataFrame
df.head()


,Title,URL
0,Alinea,https://cdn.choosechicago.com/uploads/2019/08/...
1,City Winery,https://cdn.choosechicago.com/uploads/2019/06/...
2,The Art Institute of Chicago,https://cdn.choosechicago.com/uploads/2019/08/...
3,Navy Pier,https://cdn.choosechicago.com/uploads/2019/05/...


In [15]:
import os
from PIL import Image
import requests

# Function to convert image to RGB PNG format and save to 'video_image' folder
def convert_to_rgb_png(image_path):
    img = Image.open(image_path).convert("RGB")
    png_filename = os.path.splitext(os.path.basename(image_path))[0] + ".png"
    png_path = os.path.join('video_image', png_filename)
    img.save(png_path, "PNG")
    return png_path

# Function to download images from URLs in DataFrame and save to 'video_image' folder
def download_images(df):
    for index, row in df.iterrows():
        title = row['Title'].replace(' ', '_')
        url = row['URL']
        image_path = os.path.join('video_image', f"{title}.png")

        # Download the image
        response = requests.get(url)
        if response.status_code == 200:
            with open(image_path, 'wb') as file:
                file.write(response.content)
            print(f"Downloaded: {image_path}")

            # Convert to PNG and save
            png_path = convert_to_rgb_png(image_path)
            print(f"Converted and saved: {png_path}")
        else:
            print(f"Failed to download: {image_path}")

# Example usage:
if __name__ == "__main__":

    # Create 'video_image' folder if it doesn't exist
    if not os.path.exists('video_image'):
        os.makedirs('video_image')

    # Download the images using the download_images function
    download_images(df)


Downloaded: video_image/Alinea.png
Converted and saved: video_image/Alinea.png
Downloaded: video_image/City_Winery.png
Converted and saved: video_image/City_Winery.png
Downloaded: video_image/The_Art_Institute_of_Chicago.png
Converted and saved: video_image/The_Art_Institute_of_Chicago.png
Downloaded: video_image/Navy_Pier.png
Converted and saved: video_image/Navy_Pier.png
